In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from math import sqrt

#imports locais
sns.set

<function seaborn.rcmod.set(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=True, rc=None)>

## Funções para resolução do sistema

In [2]:
def eliminacao_gauss(matriz, dimensao, b):
    x = []
    for i in range(dimensao):
        x.append(1)

    for k in range(0, dimensao - 1  ):
        i = k+1
        for i in range(k+1, dimensao ):
            m = matriz[i][k]/matriz[k][k]
            matriz[i][k] = 0
            j = k+1
            for j in range(k+1, dimensao):
                matriz[i][j] = matriz[i][j] - (m * matriz[k][j])

            b[i] = b[i] - m * b[k]


    x[dimensao - 1] = b[dimensao-1] / matriz[dimensao-1][dimensao-1]
    for i in range(dimensao-2, -1, -1):
        soma = 0
        for j in range(i+1, dimensao):
            soma = soma + matriz[i][j] * x[j]

        x[i] = (b[i] - soma)/matriz[i][i]

    return x, matriz

In [3]:
def calc_erro(x_atual,x_ant, err_list):
    num = []
    for i in range (len(x_atual)):
        num.append(abs(x_atual[i] - x_ant[i]))

    err = max(num)/abs(max(x_atual))
    err_list.append(err)
    return err


In [4]:

def gaussSeidel(matriz, b, tol , stop ):
    err_list = []
    x_ant = []
    x_atual = []
    for i in range(len(b)):
        x_ant.append(0) 
        x_atual.append(0)

    k = 1
    while(k <= stop):
        for i in range(0,len(b)):
            alpha = 0
            for j in range(0, i):
                alpha = alpha + matriz[i][j] * x_atual[j]
            for j in range(i+1, len(b)):
                alpha = alpha + matriz[i][j] * x_ant[j]
            x_atual[i] = (b[i] - alpha) / matriz[i][i]
        
        if( abs(calc_erro(x_atual,x_ant, err_list)) < tol):
            return x_atual, err_list
        for i in range(len(x_atual)):
            x_ant[i] = x_atual[i]
        k = k+1

    print('numero de iteracoes expirados')
    return x_atual, err_list


## Criando um dataframe com a base de dados

In [5]:
df = pd.read_csv('input/machine.data',  header=None)

In [6]:
df.columns = ['vendon_names','model','MYCT','MMIN', 'MMAX', 'CACH','CHMIN', 'CHMAX', 'PRP', 'ERP']

10 primeiros itens do dataset

In [7]:
df.head(10)

,vendon_names,model,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP,ERP
0,adviser,32/60,125,256,6000,256,16,128,198,199
1,amdahl,470v/7,29,8000,32000,32,8,32,269,253
2,amdahl,470v/7a,29,8000,32000,32,8,32,220,253
3,amdahl,470v/7b,29,8000,32000,32,8,32,172,253
4,amdahl,470v/7c,29,8000,16000,32,8,16,132,132
5,amdahl,470v/b,26,8000,32000,64,8,32,318,290
6,amdahl,580-5840,23,16000,32000,64,16,32,367,381
7,amdahl,580-5850,23,16000,32000,64,16,32,489,381
8,amdahl,580-5860,23,16000,64000,64,16,32,636,749
9,amdahl,580-5880,23,32000,64000,128,32,64,1144,1238


In [9]:
models = df['model']

In [8]:
df.shape
df_y = df['ERP']
df = df.drop(columns=['ERP'])
df_x = df

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df_x.values, df_y.values, test_size=0.3, random_state=1 )

In [11]:
X_train.shape

(146, 9)

In [12]:
X_train.transpose()

array([['dg', 'prime', 'dg', ..., 'hp', 'nas', 'cdc'],
       ['eclipse:mv/8000-ii', '50-750-ii', 'eclipse:mv/8000', ...,
        '3000/iii', 'as/5000', 'omega:480-ii'],
       [220, 160, 110, ..., 175, 92, 50],
       ...,
       [1, 1, 1, ..., 3, 1, 1],
       [2, 14, 2, ..., 24, 6, 5],
       [71, 60, 60, ..., 22, 62, 29]], dtype=object)

In [13]:
G_GT =  np.dot(X_train.transpose(), X_train)

TypeError: can't multiply sequence by non-int of type 'str'

In [ ]:
G_GT

In [ ]:
GT_y = np.dot(X_train.transpose(),y_train)

In [ ]:
alphas, err_list = gaussSeidel(G_GT, GT_y, 0.00005, 500000)

In [ ]:
y_predict = []
mult = []
for test in X_test:
    for i in range(len(alphas)):
        mult.append(test[i] * alphas[i])
    y_predict.append(sum(mult))
    mult.clear()

In [ ]:
y_predict

In [ ]:
error = sqrt(mean_squared_error(y_test,y_predict))

In [ ]:
error

In [ ]:
modelsdf['model'][0:146]

In [ ]:
pca = PCA(n_components=1)
X_train_pca = pca.fit_transform(X_train)
X_train_1d = []
for element in X_train_pca:
    X_train_1d.append(element[0])

In [ ]:
X_train_1d

In [ ]:
sns.regplot(x=X_train_1d, y=y_train, data=df)